In [2]:
import pandas as pd
import re
import os
from datetime import datetime

class SecondStageJobFilter:
    def __init__(self):
        # HIGH-VALUE keywords that indicate truly relevant analytical roles
        self.high_value_keywords = {
            'operations_research': [
                'operations research', 'operational research', 'or analyst', 'or specialist',
                'optimization', 'linear programming', 'mathematical modeling', 'mathematical modelling',
                'supply chain optimization', 'logistics optimization', 'decision science',
                'management science', 'operational analytics', 'process optimization'
            ],
            'advanced_analytics': [
                'data scientist', 'machine learning', 'artificial intelligence', 'ai specialist',
                'predictive analytics', 'statistical modeling', 'statistical modelling',
                'data mining', 'big data', 'advanced analytics', 'quantitative analysis',
                'econometrics', 'biostatistics', 'statistical analysis'
            ],
            'business_intelligence': [
                'business intelligence', 'bi analyst', 'data analyst', 'business analyst',
                'reporting analyst', 'analytics manager', 'insights analyst',
                'performance analyst', 'commercial analyst', 'strategy analyst',
                'planning analyst', 'research analyst', 'market research analyst'
            ],
            'financial_analytics': [
                'quantitative analyst', 'quant', 'risk analyst', 'credit analyst',
                'financial analyst', 'investment analyst', 'portfolio analyst',
                'actuarial analyst', 'pricing analyst', 'financial modeller',
                'credit risk analyst', 'market risk analyst', 'operational risk analyst'
            ],
            'research_roles': [
                'research scientist', 'researcher', 'research associate', 'research manager',
                'market researcher', 'user researcher', 'ux researcher', 'policy researcher',
                'graduate researcher', 'research director', 'research executive'
            ],
            'consulting_analytics': [
                'management consultant', 'strategy consultant', 'analytics consultant',
                'business consultant', 'data consultant', 'consulting analyst',
                'management science consultant', 'operational consultant'
            ]
        }
        
        # Combine all high-value keywords
        self.all_high_value = []
        for category, keywords in self.high_value_keywords.items():
            self.all_high_value.extend(keywords)
        
        # REFINED exclusions - jobs that shouldn't be in analytical dataset
        self.refined_exclusions = [
            # Basic administrative roles
            'hr administrator', 'hr assistant', 'hr advisor', 'hr coordinator',
            'office administrator', 'administration assistant', 'admin coordinator',
            'executive assistant', 'personal assistant', 'office manager',
            
            # Basic customer service
            'customer service', 'customer advisor', 'customer support', 'client services',
            'customer success', 'account coordinator', 'client coordinator',
            
            # Basic sales/marketing
            'sales executive', 'sales manager', 'sales advisor', 'sales representative',
            'marketing executive', 'marketing assistant', 'marketing coordinator',
            'business development executive', 'account executive',
            
            # Basic finance/accounting (non-analytical)
            'accounts assistant', 'accounts clerk', 'bookkeeper', 'finance assistant',
            'payroll administrator', 'credit controller', 'billing coordinator',
            'accounts payable', 'accounts receivable', 'purchase ledger',
            
            # Basic IT/technical support
            'it support', 'technical support', 'help desk', 'desktop support',
            'system administrator', 'network administrator', 'it administrator',
            
            # Basic operations
            'operations coordinator', 'operations assistant', 'logistics coordinator',
            'supply chain coordinator', 'procurement coordinator',
            
            # Basic recruitment/HR
            'recruitment administrator', 'recruitment coordinator', 'talent coordinator',
            'resourcing specialist', 'hr generalist', 'hr business partner',
            
            # Non-analytical legal
            'legal assistant', 'legal secretary', 'legal administrator', 'compliance officer',
            'regulatory officer', 'legal coordinator',
            
            # Basic project roles
            'project coordinator', 'project administrator', 'program coordinator',
            'project assistant', 'program assistant'
        ]
        
        # Advanced exclusion patterns
        self.advanced_exclusion_patterns = [
            r'\btrainee\s+(?!data|analyst|research)', # Trainee roles except data/analyst/research
            r'\bgraduate\s+(?!data|analyst|research|statistician)', # Graduate roles except analytical
            r'\bjunior\s+(?!data|analyst|research)', # Junior roles except analytical
            r'\bassistant\s+(?!research|data)', # Assistant roles except research/data
            r'\bcoordinator\s+(?!research|data|analytics)', # Coordinator except analytical
            r'\badministrator\s+(?!data|database)', # Administrator except data-related
            r'\bsupport\s+(?!analyst|research|data)', # Support roles except analytical
            r'\bspecialist\s+(?!data|research|analytics|risk|credit)', # Specialist except analytical
            r'\bofficer\s+(?!research|data|analytics|risk)', # Officer except analytical
        ]

    def load_filtered_dataset(self, file_path):
        """Load the already filtered dataset"""
        try:
            df = pd.read_excel(file_path)
            print(f"Loaded filtered dataset: {len(df):,} jobs")
            print(f"Columns: {list(df.columns)}")
            return df
        except Exception as e:
            print(f"Error loading dataset: {e}")
            return None

    def calculate_relevance_score(self, job_title, job_description, job_requirements):
        """Calculate a relevance score for each job"""
        score = 0
        reasons = []
        
        # Combine all text for analysis
        combined_text = f"{job_title or ''} {job_description or ''} {job_requirements or ''}".lower()
        
        # High-value keyword scoring
        for category, keywords in self.high_value_keywords.items():
            category_matches = 0
            for keyword in keywords:
                if keyword in combined_text:
                    category_matches += 1
                    reasons.append(f"{category}: {keyword}")
            
            # Score based on category importance
            if category == 'operations_research':
                score += category_matches * 5  # Highest priority
            elif category == 'advanced_analytics':
                score += category_matches * 4
            elif category == 'financial_analytics':
                score += category_matches * 3
            elif category == 'business_intelligence':
                score += category_matches * 2
            elif category in ['research_roles', 'consulting_analytics']:
                score += category_matches * 2
        
        # Bonus for job title relevance
        title_lower = (job_title or '').lower()
        high_value_title_keywords = [
            'analyst', 'scientist', 'researcher', 'quant', 'statistician',
            'econometrician', 'consultant', 'manager', 'director', 'lead'
        ]
        
        for keyword in high_value_title_keywords:
            if keyword in title_lower:
                score += 2
                reasons.append(f"title: {keyword}")
        
        # Penalty for refined exclusions
        for exclusion in self.refined_exclusions:
            if exclusion in combined_text:
                score -= 3
                reasons.append(f"PENALTY: {exclusion}")
        
        # Penalty for advanced exclusion patterns
        for pattern in self.advanced_exclusion_patterns:
            if re.search(pattern, combined_text):
                score -= 2
                match = re.search(pattern, combined_text).group()
                reasons.append(f"PENALTY: {match}")
        
        return max(0, score), reasons  # Don't allow negative scores

    def apply_second_filter(self, df, min_score=3):
        """Apply second stage filtering with scoring"""
        print(f"\nApplying second-stage filtering...")
        print(f"Minimum relevance score: {min_score}")
        
        # Calculate relevance scores
        scores = []
        score_reasons = []
        
        for idx, row in df.iterrows():
            if idx % 500 == 0:
                print(f"Processed {idx:,} of {len(df):,} jobs...")
            
            job_title = row.get('job_title', '')
            job_description = row.get('job_description', '')
            job_requirements = row.get('job_requirements', '')
            
            score, reasons = self.calculate_relevance_score(job_title, job_description, job_requirements)
            scores.append(score)
            score_reasons.append('; '.join(reasons) if reasons else 'No specific matches')
        
        # Add scores to dataframe
        df['relevance_score'] = scores
        df['score_reasons'] = score_reasons
        df['second_filter_date'] = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
        
        # Filter based on minimum score
        highly_relevant_df = df[df['relevance_score'] >= min_score].copy()
        
        print(f"\nSecond-stage filtering complete!")
        print(f"Original filtered jobs: {len(df):,}")
        print(f"Highly relevant jobs (score >= {min_score}): {len(highly_relevant_df):,}")
        print(f"Further reduction: {len(df) - len(highly_relevant_df):,} jobs removed")
        print(f"Final percentage: {len(highly_relevant_df)/len(df)*100:.1f}%")
        
        return highly_relevant_df

    def show_score_distribution(self, df):
        """Show distribution of relevance scores"""
        print("\n" + "="*60)
        print("RELEVANCE SCORE DISTRIBUTION")
        print("="*60)
        
        score_counts = df['relevance_score'].value_counts().sort_index()
        for score, count in score_counts.items():
            print(f"Score {score}: {count:,} jobs")
        
        print(f"\nAverage score: {df['relevance_score'].mean():.2f}")
        print(f"Median score: {df['relevance_score'].median():.2f}")
        print(f"Max score: {df['relevance_score'].max()}")

    def show_top_jobs(self, df, n=10):
        """Show top-scoring jobs"""
        print("\n" + "="*60)
        print(f"TOP {n} HIGHEST-SCORING JOBS")
        print("="*60)
        
        top_jobs = df.nlargest(n, 'relevance_score')[['job_title', 'company_name', 'relevance_score', 'score_reasons']]
        
        for idx, row in top_jobs.iterrows():
            print(f"\nScore: {row['relevance_score']}")
            print(f"Title: {row['job_title']}")
            print(f"Company: {row['company_name']}")
            print(f"Reasons: {row['score_reasons'][:100]}...")

    def save_refined_data(self, df, output_path):
        """Save the refined dataset"""
        try:
            # Sort by relevance score (highest first)
            df_sorted = df.sort_values('relevance_score', ascending=False)
            
            with pd.ExcelWriter(output_path, engine='openpyxl') as writer:
                df_sorted.to_excel(writer, sheet_name='Highly_Relevant_Jobs', index=False)
            
            print(f"\nRefined data saved to: {output_path}")
            return True
        except Exception as e:
            print(f"Error saving refined data: {e}")
            return False

def main():
    """Main function for second-stage filtering"""
    
    print("="*70)
    print("SECOND STAGE FILTER - REFINE ANALYTICAL JOBS")
    print("="*70)
    print("This tool applies advanced scoring to identify the most relevant")
    print("analytical, research, and data science positions.")
    print("="*70)
    
    # Pre-configured paths
    input_path = r"C:\Users\HP\OneDrive - University of Southampton\Documents\Dissertation Project - Marwa Ashfaq\Dataset\50K_jobs_filtered.xlsx"
    output_path = r"C:\Users\HP\OneDrive - University of Southampton\Documents\Dissertation Project - Marwa Ashfaq\Dataset\50K_jobs_highly_relevant.xlsx"
    
    print(f"Input file: {input_path}")
    print(f"Output file: {output_path}")
    
    # Initialize filter
    filter_tool = SecondStageJobFilter()
    
    # Load filtered dataset
    df = filter_tool.load_filtered_dataset(input_path)
    if df is None:
        return
    
    # Show current dataset info
    print(f"\nDataset overview:")
    print(f"Total jobs: {len(df):,}")
    if 'job_title' in df.columns:
        print(f"Sample job titles:")
        for title in df['job_title'].head(10):
            print(f"  - {title}")
    
    # Apply second-stage filtering with scoring
    print(f"\nStarting relevance scoring...")
    refined_df = filter_tool.apply_second_filter(df, min_score=3)
    
    # Show score distribution
    filter_tool.show_score_distribution(refined_df)
    
    # Show top jobs
    filter_tool.show_top_jobs(refined_df)
    
    # Ask if user wants to adjust the minimum score
    print(f"\n" + "="*60)
    print("SCORE THRESHOLD ADJUSTMENT")
    print("="*60)
    current_count = len(refined_df)
    print(f"Current results with score >= 3: {current_count:,} jobs")
    
    adjust = input("Do you want to try a different minimum score? (y/n): ").strip().lower()
    
    if adjust in ['y', 'yes']:
        try:
            new_score = int(input("Enter new minimum score (1-10): "))
            if 1 <= new_score <= 10:
                refined_df = df[df['relevance_score'] >= new_score].copy()
                print(f"New results with score >= {new_score}: {len(refined_df):,} jobs")
            else:
                print("Invalid score. Using original results.")
        except ValueError:
            print("Invalid input. Using original results.")
    
    # Save refined data
    if len(refined_df) > 0:
        success = filter_tool.save_refined_data(refined_df, output_path)
        
        if success:
            print("\n" + "="*70)
            print("SECOND-STAGE FILTERING COMPLETED!")
            print("="*70)
            print(f"Original filtered jobs: {len(df):,}")
            print(f"Highly relevant jobs: {len(refined_df):,}")
            print(f"Final reduction: {((len(df) - len(refined_df))/len(df)*100):.1f}%")
            print(f"Quality improvement: Higher concentration of analytical roles")
            print(f"Saved to: {output_path}")
            
            # Show category breakdown if available
            if 'category' in refined_df.columns:
                print(f"\nTop categories in refined results:")
                category_counts = refined_df['category'].value_counts().head(10)
                for category, count in category_counts.items():
                    print(f"  {category}: {count}")
        else:
            print("Failed to save refined data.")
    else:
        print("No jobs met the minimum score criteria. Try lowering the minimum score.")

if __name__ == "__main__":
    main()

SECOND STAGE FILTER - REFINE ANALYTICAL JOBS
This tool applies advanced scoring to identify the most relevant
analytical, research, and data science positions.
Input file: C:\Users\HP\OneDrive - University of Southampton\Documents\Dissertation Project - Marwa Ashfaq\Dataset\50K_jobs_filtered.xlsx
Output file: C:\Users\HP\OneDrive - University of Southampton\Documents\Dissertation Project - Marwa Ashfaq\Dataset\50K_jobs_highly_relevant.xlsx
Loaded filtered dataset: 2,434 jobs
Columns: ['category', 'city', 'company_name', 'geo', 'job_board', 'job_description', 'job_requirements', 'job_title', 'job_type', 'post_date', 'salary_offered', 'state', 'keywords_found', 'filter_date']

Dataset overview:
Total jobs: 2,434
Sample job titles:
  - Business Manager - Lexus 
  - Supply Chain Business Analyst
  - SQL Insight Analyst
  - Production Design Engineer
  - HR Advisor
  - HRIS Analyst - Move Into Workday - Work From Home
  - Design Estimator
  - Tax Specialist - Private Client
  - Senior Confe

Do you want to try a different minimum score? (y/n):  n



Refined data saved to: C:\Users\HP\OneDrive - University of Southampton\Documents\Dissertation Project - Marwa Ashfaq\Dataset\50K_jobs_highly_relevant.xlsx

SECOND-STAGE FILTERING COMPLETED!
Original filtered jobs: 2,434
Highly relevant jobs: 1,053
Final reduction: 56.7%
Quality improvement: Higher concentration of analytical roles
Saved to: C:\Users\HP\OneDrive - University of Southampton\Documents\Dissertation Project - Marwa Ashfaq\Dataset\50K_jobs_highly_relevant.xlsx

Top categories in refined results:
  banking jobs: 159
  it jobs: 119
  construction property jobs: 115
  strategy consultancy jobs: 107
  marketing jobs: 97
  science jobs: 59
  graduate training internships jobs: 57
  finance jobs: 47
  accountancy qualified jobs: 43
  energy jobs: 29
